In [1]:
import numpy as np
import h5py

import pylandau

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Circle

from scipy.optimize import minimize
from scipy.stats import poisson

import torch
import torch.nn.functional as F

In [1]:
!ls -l /global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan0000001544.h5

-rw-r--r-- 1 percius ncemhub 10957890448 Oct  2  2023 /global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan0000001544.h5


In [2]:
!h5ls -lvr counted_halo_scan0000000159.h5

Opened "counted_halo_scan0000000159.h5" with sec2 driver.
/                        Group
    Location:  1:96
    Links:     1
/electron_events         Group
    Location:  1:800
    Links:     1
/electron_events/scan_positions Dataset {1049600/1049600}
    Attribute: Nx scalar
        Type:      native long
        Data:  1025
    Attribute: Ny scalar
        Type:      native long
        Data:  1024
    Location:  1:3403823760
    Links:     1
    Storage:   8396800 logical bytes, 8396800 allocated bytes, 100.00% utilization
    Type:      native long
/electron_events/weights Dataset {1049600/1049600}
    Location:  1:2432
    Links:     1
    Storage:   information not available
    Type:      variable length of
                   native int
/metadata                Group
    Location:  1:3403824032
    Links:     1
/metadata/electron_counting Group
    Attribute: background_threshold scalar
        Type:      native double
        Data:  28.5471
    Attribute: background_threshold_

In [ ]:
!nvidia-smi

In [ ]:
#!pip install pylandau
#!pip install DuSC_explorer

In [ ]:
# Does not work in Jupyter (must run from terminal)
#import DuSC_explorer
#DuSC_explorer.open_file()

## Fit-based counting algorithm

In [2]:
nframes = 1049600 # 2064

In [3]:
def gaussian_splash_pytorch(A, sigma, size=3, device='cpu'):
    """Prepare the Gaussian splash as a PyTorch convolution kernel."""
    x = y = torch.arange(0, size, device=device) - (size // 2)
    X, Y = torch.meshgrid(x, y)
    gaussian = A * torch.exp(- (X**2 + Y**2) / (2 * sigma**2))
    # Reshape to 4D tensor: (out_channels, in_channels, height, width)
    gaussian_kernel = gaussian.unsqueeze(0).unsqueeze(0)
    return gaussian_kernel

def construct_modeled_frame_pytorch(frame_ct, splash_kernel, noise_sigma, device='cpu'):
    """Construct the modeled frame using convolution to apply the Gaussian splash."""
    # Ensure frame_ct is a float tensor and add batch and channel dimensions
    frame_ct_tensor = frame_ct.float().unsqueeze(1)
    
    # Apply the Gaussian splash across the entire frame using convolution
    modeled_frame = F.conv2d(frame_ct_tensor, splash_kernel, padding=splash_kernel.shape[-1]//2)
    
    # Add noise
    #modeled_frame = modeled_frame + torch.normal(0, noise_sigma, size=frame.shape, device=device)
    
    # Remove batch and channel dimensions from the output
    modeled_frame = modeled_frame.squeeze(1)
    
    return modeled_frame

def count_frame_pytorch(frame_bls, frame_ct, n_steps_max=500, loss_lim = 1):
    """Counts a frame given initial guess frame_ct"""
    
    # Convert frame and frame_ct to a PyTorch tensor
    frame_tensor = torch.from_numpy(frame_bls).to(device)
    frame_ct_tensor = torch.tensor(frame_ct, dtype=torch.float32, device=device, requires_grad=True)

    # Define the optimizer
    optimizer = torch.optim.Adam([frame_ct_tensor], lr=0.01)

    # Define the single-electron Gaussian splash
    splash = gaussian_splash_pytorch(A,sigma,device=device)

    # Set up the parameters for the iteration
    n_steps = 0
    loss = loss_lim + 1
    
    while(loss > loss_lim and n_steps < n_steps_max):
        
        optimizer.zero_grad()  # Clear previous gradients

        # Construct the modeled frame
        modeled_frame = construct_modeled_frame_pytorch(frame_ct_tensor, splash, noise_sigma, device=device)

        # Compute the loss (negative likelihood)
        loss = torch.sum((frame_tensor - modeled_frame) ** 2)

        # Compute gradients
        loss.backward()

        # Update frame_ct_tensor based on gradients
        optimizer.step()

        n_steps += 1
        if(n_steps >= n_steps_max):
            print(f"Warning, stopping at max steps {n_steps} with loss {loss}")
        if n_steps % 100 == 0:
            print(f"Step {n_steps}, Loss: {loss.item()}")
    print(f"Counted in n_steps = {n_steps} with loss = {loss}")
    
    return frame_ct_tensor.cpu().detach().numpy(), modeled_frame.cpu().detach().numpy()

def frame_to_indices_weights(counted_frames):
    """Convert a batch of 2D counted frames into lists of linear indices and weights."""
    batch_size = counted_frames.shape[0]
    frame_shape = counted_frames.shape[1:]
    all_linear_indices = []
    all_weights = []

    for i in range(batch_size):
        frame = counted_frames[i]
        nonzero_indices = np.nonzero(frame)
        weights = frame[nonzero_indices]
        linear_indices = np.ravel_multi_index(nonzero_indices, frame_shape)
        
        all_linear_indices.append(linear_indices)
        all_weights.append(weights)

    return all_linear_indices, all_weights

def update_counted_data_hdf5(file_path, batch_start_idx, frames_indices, frames_weights, group_name='electron_events'):
    # Ensure frames_indices and frames_weights are lists of arrays, 
    # with each array corresponding to one frame's data.
    
    with h5py.File(file_path, 'a') as f:  # Open file in append mode
        # Create or access the group
        if group_name not in f:
            grp = f.create_group(group_name)
        else:
            grp = f[group_name]

        # Check if the VL datasets exist, create them if not
        if 'frames' not in grp:
            vl_dtype_indices = h5py.special_dtype(vlen=np.dtype('uint32'))
            vl_dataset_indices = grp.create_dataset("frames", (nframes,), dtype=vl_dtype_indices)
        else:
            vl_dataset_indices = grp['frames']

        if 'weights' not in grp:
            vl_dtype_weights = h5py.special_dtype(vlen=np.dtype('uint32'))
            vl_dataset_weights = grp.create_dataset("weights", (nframes,), dtype=vl_dtype_weights)
        else:
            vl_dataset_weights = grp['weights']

        # Assuming the length of frames_indices matches the expected number of frames,
        # iterate through each and update the datasets
        for i, (indices, weights) in enumerate(zip(frames_indices, frames_weights)):
            vl_dataset_indices[batch_start_idx+i] = indices
            vl_dataset_weights[batch_start_idx+i] = weights
            
def compute_conditional_probabilities(lam_grid):
    """
    Compute the array of conditional probabilities P(n >= 2) / P(n >= 1) for a grid of lambda values.
    
    :param lam_grid: A 576x576 numpy array of lambda values.
    :return: A 576x576 numpy array of conditional probabilities.
    """
    # Compute P(n >= 1) and P(n >= 2) for each lambda in the grid
    p_at_least_1 = 1 - poisson.cdf(0, lam_grid)  # P(n >= 1) = 1 - P(n < 1)
    p_at_least_2 = 1 - poisson.cdf(1, lam_grid)  # P(n >= 2) = 1 - P(n <= 1)
    
    # Conditional probability P(n >= 2) / P(n >= 1)
    # Safeguard division by zero by using np.where to only compute valid divisions
    conditional_prob = np.where(p_at_least_1 > 0, p_at_least_2 / p_at_least_1, 0)
    
    return conditional_prob

In [4]:
# Initial variables
dataset_path = "/pscratch/sd/j/jrenner1/data_scan0000000159.h5" # "/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan0000001544.h5"
counted_data_path = "counted_halo_scan0000000159.h5" #"counted_phase2_scan0000001544.h5"
A = 19.032279882831432
sigma = 0.5065158792172835
noise_sigma = 1.0
device='cuda'
frame_width = 576
nbatch = 2064
baseline = 20.0
th_single_elec = 10

##### Prior determination (skip this to operate without prior)

In [ ]:
# Create the "prior"
with h5py.File(dataset_path, 'r') as f0:
    data = f0['frames']
    
    # Get all frames and subtract the baseline.
    prior_bls = np.array(data[0:nframes,:,:],dtype=np.float32) - baseline
    print(f"Prior values, {nframes} frames, shape: {prior_bls.shape}")
    
# Compute the summed frame.
prior_frame = np.sum(prior_bls,axis=0)
print("Summed frame dimensions:",prior_frame.shape)

# Divide by the average electron amplitude.
prior_frame /= A

# Normalize by the number of frames to get the final "prior".
prior_frame /= nframes

# Eliminate negative values.
prior_frame[prior_frame < 0] = 0.

# Compute the conditional probabilities of having >= 2 counts, given >= 1 count.
conditional_prob = compute_conditional_probabilities(prior_frame)

In [ ]:
# Plot the prior
plt_frame = prior_frame

iprior = plt.imshow(plt_frame, vmin=plt_frame.min(), vmax=plt_frame.max(), cmap='gray')
plt.colorbar()
#axs[i].set_title(f'Scan No: {scan_no}')

In [ ]:
# Repeat the conditional probability matrix nbatch times.
conditional_prob_batch = np.repeat(conditional_prob[np.newaxis,:,:], nbatch, axis=0)

In [ ]:
random_matrix = np.random.rand(nbatch,frame_width,frame_width)
single_electron_pixels = random_matrix > conditional_prob_batch

In [ ]:
n_single_elec = np.sum(single_electron_pixels)
n_total = nbatch*frame_width*frame_width
print(f"TEST: {n_single_elec} of {n_total} ({n_single_elec/n_total*100:.5f}%) were single-electrons")

##### Perform the counting

In [6]:
batches = round(nframes / nbatch)
print(f"Analyzing in {batches} batches...")
for batch in range(batches)[494:]:
    print(f"\n\n ** BATCH {batch} **")
    
    # Get the frames for this batch
    print("-- Processing frames...")
    with h5py.File(dataset_path, 'r') as f0:
        data = f0['frames']

        # Get the frames
        frame_bls = np.array(data[batch*nbatch:batch*nbatch+nbatch,:,:],dtype=np.float32) - baseline

        # Compute an initial counted frame
        frame_ct = np.rint(frame_bls / A, out=np.zeros(frame_bls.shape,dtype=np.int16), casting='unsafe')
        frame_ct[frame_ct < th_single_elec] = 0

    # -------------------------------------------------------------------------------
    # Count the frames.
    print("-- Counting frames...")
    frame_ct_reco, modeled_frame_reco = count_frame_pytorch(frame_bls, frame_ct, n_steps_max = 300, loss_lim = nframes)
    frame_ct_reco[frame_ct_reco < 0] = 0
    frame_ct_reco = np.rint(frame_ct_reco)
    # -------------------------------------------------------------------------------
    
    # -------------------------------------------------------------------------------
    # Apply the prior for each frame.
    # Generate a 576x576 matrix of random numbers from a uniform distribution [0, 1)
#     random_matrix = np.random.rand(*frame_ct_reco.shape)

#     # Compare the random matrix to the probability matrix:
#     #  - if the random number is greater than the conditional probability, 
#     #    the multi-count is said to have been due to the Landau
#     #    tail, and therefore the count is set to 1
#     single_electron_pixels = random_matrix > conditional_prob_batch

#     # Set all counts > 1 that did not pass the probability game to 1 count.
#     forced_single_electrons = (frame_ct_reco > 1) & single_electron_pixels
#     n_single_elec = np.sum(forced_single_electrons)
#     n_total = nbatch*frame_width*frame_width
#     print(f"{n_single_elec} of {n_total} ({n_single_elec/n_total*100:.4f}%) forced to single-electron counts")
#     frame_ct_reco[forced_single_electrons] = 1
    # -------------------------------------------------------------------------------
    
    # Save the counted frames in the array
    print("-- Saving frames...")
    frames_indices, frames_weights = frame_to_indices_weights(frame_ct_reco)
    print(f"Frame indices len = {len(frames_indices)} and weights = {len(frames_weights)}")
    update_counted_data_hdf5(counted_data_path, batch*nbatch, frames_indices, frames_weights)

Analyzing in 509 batches...


 ** BATCH 494 **
-- Processing frames...
-- Counting frames...


/global/common/software/nersc/pm-2022q4/sw/pytorch/2.0.1/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pscratch/sd/s/swowner/pytorch-build/pytorch/2.0.1/pytorch/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Step 100, Loss: 1387016960.0
Step 200, Loss: 1034559872.0
Warning, stopping at max steps 300 with loss 787358400.0
Step 300, Loss: 787358400.0
Counted in n_steps = 300 with loss = 787358400.0
-- Saving frames...
Frame indices len = 2064 and weights = 2064


 ** BATCH 495 **
-- Processing frames...
-- Counting frames...
Step 100, Loss: 1379265792.0
Step 200, Loss: 1028332800.0
Warning, stopping at max steps 300 with loss 782325376.0
Step 300, Loss: 782325376.0
Counted in n_steps = 300 with loss = 782325376.0
-- Saving frames...
Frame indices len = 2064 and weights = 2064


 ** BATCH 496 **
-- Processing frames...
-- Counting frames...
Step 100, Loss: 1382154496.0
Step 200, Loss: 1030253056.0
Warning, stopping at max steps 300 with loss 783710208.0
Step 300, Loss: 783710208.0
Counted in n_steps = 300 with loss = 783710208.0
-- Saving frames...
Frame indices len = 2064 and weights = 2064


 ** BATCH 497 **
-- Processing frames...
-- Counting frames...
Step 100, Loss: 1380526848.0
Step 200

In [10]:
# Paths to the source and destination HDF5 files
source_file_path = '/global/cfs/cdirs/ncemhub/distiller/counted/2024.05.09/FOURD_240509_1311_16228_00159.h5'
destination_file_path = 'counted_halo_scan0000000159.h5'

def copy_item(name, obj, dest_file):
    """Recursive function to copy items and attributes from source to destination, excluding dataset data for specific paths."""
    if isinstance(obj, h5py.Dataset):
        if name == 'electron_events/frames':
            # Copy only attributes for the frames dataset
            if name not in dest_file:
                # If dataset doesn't exist, create an empty one just to hold attributes
                print(f"Copying dataset {name}")
                dest_file.create_dataset(name, shape=(0,), dtype=obj.dtype)
            # Copy attributes
            print(f"Copying attributes for {name}")
            for attr_name, attr_value in obj.attrs.items():
                dest_file[name].attrs[attr_name] = attr_value
        elif name == 'electron_events/scan_positions':
            # Overwrite or recreate scan_positions dataset with all frames
            if name in dest_file:
                del dest_file[name]  # Delete the existing dataset to recreate it
            print(f"Copying all data for {name}")
            dest_dataset = dest_file.create_dataset(name, data=obj[:], dtype=obj.dtype)
            for attr_name, attr_value in obj.attrs.items():
                dest_dataset.attrs[attr_name] = attr_value
        else:
            # Handle other datasets
            if name not in dest_file or name == 'electron_events/scan_positions':
                # Calculate the size to copy: minimum of dataset size or nframes
                size_to_copy = min(obj.size, nframes)
                # Copy the dataset with the limited size
                dest_dataset = dest_file.create_dataset(name, data=obj[:size_to_copy], dtype=obj.dtype)
                # Copy attributes for the dataset
                for attr_name, attr_value in obj.attrs.items():
                    dest_dataset.attrs[attr_name] = attr_value
    elif isinstance(obj, h5py.Group):
        # Create group if it doesn't exist
        if name not in dest_file:
            dest_file.create_group(name)
        # Copy attributes for the group
        for attr_name, attr_value in obj.attrs.items():
            dest_file[name].attrs[attr_name] = attr_value

# Open both the source and destination file
with h5py.File(source_file_path, 'r') as source, h5py.File(destination_file_path, 'a') as dest:
    # Walk through the source file and copy everything, applying specific rules to frames dataset
    source.visititems(lambda name, obj: copy_item(name, obj, dest))

print("Data and attributes have been copied successfully, with specific handling for /electron_events/frames attributes.")

Copying attributes for electron_events/frames
Copying all data for electron_events/scan_positions
Data and attributes have been copied successfully, with specific handling for /electron_events/frames attributes.


In [5]:
# Replace int32->int64
destination_file_path = 'counted_halo_scan0000000159.h5'

# Open the HDF5 file in read/write mode
with h5py.File(destination_file_path, 'r+') as file:
    # Access the dataset
    dataset = file['/electron_events/frames']

    # Check the type and structure
    print("Data dtype:", dataset.dtype)
    #print("Data is a variable-length dataset:", isinstance(dataset.dtype, h5py.vlen_dtype))

    # Prepare for the new dataset creation by copying attributes and deleting the old dataset
    old_attrs = dict(dataset.attrs)
    del file['/electron_events/frames']
    file.flush()  # Ensure the dataset is deleted before recreating

    # Create a new dataset with variable-length uint32 arrays
    # Note: Properly configure dtype for variable-length if necessary
    dtype_uint32_vlen = h5py.special_dtype(vlen=np.dtype('uint32'))
    frames_uint32 = file['/electron_events'].create_dataset('frames', shape=dataset.shape, dtype=dtype_uint32_vlen)

    # Iterate over each array, convert and write back
    for i in range(len(dataset)):
        # Convert each array individually
        array_uint32 = np.array(dataset[i], dtype=np.uint32)
        frames_uint32[i] = array_uint32

    # Reapply the saved attributes to the new dataset
    for attr_name, attr_value in old_attrs.items():
        frames_uint32.attrs[attr_name] = attr_value

    print("All data converted and written back successfully.")

Data dtype: object
All data converted and written back successfully.


### Counting analysis

In [ ]:
# Function to convert linear indices to 2D (row, col) indices
def linear_to_2d_indices(linear_indices, cols):
    rows = linear_indices // cols
    cols = linear_indices % cols
    return rows, cols

# Function to create a 2D image from the counted frame data
def create_counted_image_weighted(frame_data, weight_data, frame_height, frame_width):
    # Initialize a 2D array with zeros
    image = np.zeros((frame_height, frame_width), dtype=np.int32)

    # Convert linear indices to 2D indices
    rows, cols = linear_to_2d_indices(frame_data, frame_width)

    # Set the hit pixels
    for r,c,w in zip(rows,cols,weight_data):
        image[r, c] = w

    return image

def create_circular_mask(h, w, center=None, radius=None):
    if center is None:  # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None:  # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

def calculate_center_of_mass(counted_image):
    """Calculate the center of mass of the counted image."""
    total_mass = np.sum(counted_image)
    indices = np.indices(counted_image.shape)
    mean_x = np.sum(indices[1] * counted_image) / total_mass
    mean_y = np.sum(indices[0] * counted_image) / total_mass
    sigma_x = np.sqrt(np.sum((indices[1] - mean_x)**2 * counted_image) / total_mass)
    sigma_y = np.sqrt(np.sum((indices[0] - mean_y)**2 * counted_image) / total_mass)
    return mean_x, mean_y, sigma_x, sigma_y

def cartesian_to_polar_counts(image, center=None):
    if center is None:
        center = (image.shape[1] // 2, image.shape[0] // 2)  # Use the middle of the image
    
    y, x = np.indices(image.shape)
    x = x - center[0]
    y = y - center[1]
    
    # Convert Cartesian coordinates to polar
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    
    # Convert theta from radians to degrees
    theta = np.degrees(theta)
    theta[theta < 0] += 360  # Convert negative angles to positive angles
    
    return r, theta, image

def plot_counts_vs_angle_binned(image, target_radius, center=None, delta_radius=5, num_bins=360):
    r, theta, counts = cartesian_to_polar_counts(image, center)
    
    # Find indices where radius is close to target radius
    mask = (r >= target_radius - delta_radius) & (r <= target_radius + delta_radius)
    actual_radius = np.mean(r[mask])

    # Bin the theta values
    bins = np.linspace(0, 360, num_bins + 1)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    digitized = np.digitize(theta[mask], bins) - 1  # Indices of the bins to which each value belongs

    # Aggregate counts in each bin
    binned_counts = np.zeros(num_bins)
    np.add.at(binned_counts, digitized, counts[mask])  # Sum counts falling into each bin

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(bin_centers, binned_counts, marker='o', linestyle='-', color='b')
    plt.title(f'Count Distribution Near Radius ~{actual_radius:.2f}')
    plt.xlabel('Angle (degrees)')
    plt.ylabel('Counts')
    plt.grid(True)
    plt.show()

In [ ]:
file_path_0 = "counted_halo_scan0000000159_10000.h5"
file_path_1 = "counted_halo_scan0000000159_300.h5"

# Set up the figure
fig, axs = plt.subplots(1, 3, figsize=(8, 2))

iframe = 0
with_weights = False


with h5py.File(file_path_0, 'r') as f0:

    frame_data_0 = np.array(f0['electron_events']['frames'][iframe])
    if(with_weights):
        weight_data_0 = np.array(f0['electron_events']['weights'][iframe])
    else:
        weight_data_0 = np.ones(len(f0['electron_events']['frames'][iframe]))

    # Create a 2D image of the counted frame
    counted_image_0 = create_counted_image_weighted(frame_data_0, weight_data_0, frame_width, frame_width)

    # Plot the summed, counted image
    ax0 = axs[0].imshow(counted_image_0, vmin=counted_image_0.min(), vmax=counted_image_0.max(), cmap='gray')
    fig.colorbar(ax0, ax=axs[0])
    
with h5py.File(file_path_1, 'r') as f1:

    frame_data_1 = np.array(f1['electron_events']['frames'][iframe])
    if(with_weights):
        weight_data_1 = np.array(f1['electron_events']['weights'][iframe])
    else:
        weight_data_1 = np.ones(len(f1['electron_events']['frames'][iframe]))

    # Create a 2D image of the counted frame
    counted_image_1 = create_counted_image_weighted(frame_data_1, weight_data_1, frame_width, frame_width)

    # Plot the summed, counted image
    ax1 = axs[1].imshow(counted_image_1, vmin=counted_image_1.min(), vmax=counted_image_1.max(), cmap='gray')
    fig.colorbar(ax1, ax=axs[1])
    
counted_image_diff = counted_image_1 - counted_image_0
ax2 = axs[2].imshow(counted_image_diff, vmin=counted_image_diff.min(), vmax=counted_image_diff.max(), cmap='gray')
fig.colorbar(ax2, ax=axs[2])
n0 = len(counted_image_0[np.nonzero(counted_image_0)])
ndiff = len(counted_image_diff[np.nonzero(counted_image_diff)])
print(f"{ndiff}/{n0} = {ndiff/n0*100:.2f}% different counts")
    

In [ ]:
nframes = 2064
frame_width = 576
with_weights = False

In [ ]:
file_path = f"counted_halo_scan0000000159.h5"
#file_path = f"/global/cfs/cdirs/ncemhub/distiller/counted/2023.09.18/data_scan{scan_no}_id{idbase[igroup]+i}_electrons.h5"

# Set up the figure
fig, axs = plt.subplots(1, 1, figsize=(10, 8))

counted_electrons, counted_nelec = [], []
with h5py.File(file_path, 'r') as f0:

    summed_counted_image = np.zeros((frame_width, frame_width), dtype=np.int32)

    # Create lists for CM computations.
    mean_x_list, mean_y_list, sigma_x_list, sigma_y_list = [], [], [], []

    nelec = 0
    for iframe in range(nframes):
        #if(iframe % int(nframes/10) == 0):
        #    print(f"Processing frame {iframe}/{nframes}")
        frame_data = np.array(f0['electron_events']['frames'][iframe])
        if(with_weights):
            weight_data = np.array(f0['electron_events']['weights'][iframe])
        else:
            weight_data = np.ones(len(f0['electron_events']['frames'][iframe]))

        # Create a 2D image of the counted frame
        counted_image = create_counted_image_weighted(frame_data, weight_data, frame_width, frame_width)

        # Sum the counted images
        summed_counted_image += counted_image
        nelec += len(frame_data)

    # Sum all the counts.
    total_counts = np.sum(summed_counted_image)
    print(f"-- Total counts: {total_counts}")
    print(f"-- Total nelec:  {nelec}")
    counted_electrons.append(total_counts)
    counted_nelec.append(nelec)

    # Plot the summed, counted image
    imax = axs.imshow(summed_counted_image[250:280,265:295], vmin=summed_counted_image.min(), vmax=summed_counted_image.max(), cmap='gray')
    #imax = axs.imshow(summed_counted_image, vmin=summed_counted_image.min(), vmax=summed_counted_image.max(), cmap='gray')

In [ ]:
file_path = f"/global/cfs/cdirs/ncemhub/distiller/counted/2023.09.18/data_scan1531_id12284_electrons.h5"
with h5py.File(file_path, 'r') as f0:
    frame_data = np.array(f0['electron_events']['frames'][2])
    print(f0['electron_events']['frames'])
    print(frame_data.shape)

In [ ]:
with h5py.File("counted_scan0000001532.h5", 'r') as f0:
    frame_data = np.array(f0['electron_events']['frames'][16505])
    print(f0['electron_events']['frames'])
    print(frame_data)
    print(len(frame_data))

Create the initial prediction for the counted frame

In [ ]:
nreco = 0

with h5py.File(dataset_path, 'r') as f0:
    data = f0['frames']
    
    # Get the frame
    frame_bls = np.array(data[nreco:nreco+nframes,:,:],dtype=np.float32) - baseline
    
    # Subtract the baseline
    #frame_bls = (frame - baseline)
    
    # Get a "prior" counted frame
    #frame_ct = (frame - baseline)
    frame_ct = np.rint(frame_bls / A, out=np.zeros(frame_bls.shape,dtype=np.int16), casting='unsafe')
    frame_ct[frame_ct < th_single_elec] = 0


fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs = axs.flatten()

# Plotting on the ith subplot
ax0 = axs[0].imshow(frame_bls[0], vmin=frame_bls[0].min(), vmax=frame_bls[0].max())
fig.colorbar(ax0, ax=axs[0])
ax1 = axs[1].imshow(frame_ct[0], vmin=frame_ct[0].min(), vmax=frame_ct[0].max())
fig.colorbar(ax1, ax=axs[1])

# Print nonzero prior values
#print("Nonzero values in frame_ct:")
#print(frame_ct[np.nonzero(frame_ct)])
#np.sum(frame_ct[np.nonzero(frame_ct)])

Perform the fit

In [ ]:
frame_ct_reco, modeled_frame_reco = count_frame_pytorch(frame_bls, frame_ct, n_steps_max = 500, loss_lim = nframes)

In [ ]:
plt.imshow(modeled_frame_reco[0][100:150,100:150])

In [ ]:
frame_ct_reco[frame_ct_reco < 0] = 0
frame_ct_reco = np.rint(frame_ct_reco)

In [ ]:
nframe = 0
rng_start, rng_end = 150, 200

# Set up the figure
fig, axs = plt.subplots(1, 3, figsize=(18, 4))
axs = axs.flatten()

frame_plt = frame_bls

ax0 = axs[0].imshow(frame_plt[nframe,rng_start:rng_end,rng_start:rng_end], vmin=frame_plt[nframe,rng_start:rng_end,rng_start:rng_end].min(), vmax=frame_plt[nframe,rng_start:rng_end,rng_start:rng_end].max())
fig.colorbar(ax0, ax=axs[0])
axs[0].set_title("Frame (baseline subtracted)")
ax1 = axs[1].imshow(frame_ct_reco[nframe,rng_start:rng_end,rng_start:rng_end], vmin=frame_ct_reco[nframe,rng_start:rng_end,rng_start:rng_end].min(), vmax=frame_ct_reco[nframe,rng_start:rng_end,rng_start:rng_end].max())
fig.colorbar(ax1, ax=axs[1])
axs[1].set_title("Reconstructed counts")
ax2 = axs[2].imshow(frame_ct[nframe,rng_start:rng_end,rng_start:rng_end], vmin=frame_ct[nframe,rng_start:rng_end,rng_start:rng_end].min(), vmax=frame_ct[nframe,rng_start:rng_end,rng_start:rng_end].max())
fig.colorbar(ax2, ax=axs[2])
axs[2].set_title("Prior counts")

print(f"Total counts reco = {np.sum(frame_ct_reco[nframe,rng_start:rng_end,rng_start:rng_end])}, prior = {np.sum(frame_ct[nframe,rng_start:rng_end,rng_start:rng_end])}")

In [ ]:
#np.unique(frame_ct_reco[np.nonzero(frame_ct_reco)])
np.unique(frame_ct[np.nonzero(frame_ct)])

# Get single-electron Gaussian profile

In [ ]:
def randomized_scan_order(nrows, ncols):
    indices = [(i, j) for i in range(1, nrows-1) for j in range(1, ncols-1)]
    np.random.shuffle(indices)
    return indices

def extract_3x3_patches(original_frames, processed_frames):
    """
    Extract 3x3 patches from original frames based on hits identified in processed frames.
    
    Args:
    - original_frames: numpy array of original frames (non-baseline-subtracted, non-thresholded).
    - processed_frames: numpy array of frames after baseline subtraction and thresholding.
    
    Returns:
    - numpy array of 3x3 patches centered around identified hits.
    """
    hit_patches = []
    nrows, ncols = processed_frames.shape[1], processed_frames.shape[2]  # Frame dimensions

    for frame_index in range(processed_frames.shape[0]):
        processed_frame = processed_frames[frame_index]
        original_frame = original_frames[frame_index]
        
        # Iterate through all pixels in the processed frame to find hits
        indices = randomized_scan_order(nrows, ncols)
        for i, j in indices:
            if processed_frame[i, j] > 0:  # Center of a hit in processed frame
                
                # Extract the corresponding 3x3 patch from the original frame
                patch = original_frame[i-1:i+2, j-1:j+2]
                hit_patches.append(patch)

    return np.array(hit_patches)

In [ ]:
base_dir = "/pscratch/sd/j/jrenner1"
#base_dir = "/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18"
scan_no = "0159" #1531
nframes = 1000 #16512
frame_width = 576
baseline = 20.0
th_single_elec = 10

file_path = f"{base_dir}/data_scan000000{scan_no}.h5"

In [ ]:
# Histogram some frames to verify the baseline
with h5py.File(file_path, 'r') as f0:
    data = f0['frames']
    print("Frames array shape:",data.shape)
    
    data_hist = np.array(data[0:nframes,:,:],dtype=np.float32)
    
    plt.hist(data_hist.flatten(), bins=55, range=[0,150])
    plt.yscale('log')
    plt.xlabel("pixel value")

In [ ]:
print(f"Processing scan {scan_no}...")
with h5py.File(file_path, 'r') as f0:
    data = f0['frames']
    frames = data[0:nframes, :, :]
    
    # Subtract baseline only.
    sub_frames = (frames - baseline)

    # Subtract the baseline and threshold.
    sub_frames_th = (frames - baseline)
    sub_frames_th[sub_frames < th_single_elec] = 0

    # Extract the 3x3 Gaussian pattern
    hit_patches = extract_3x3_patches(sub_frames, sub_frames_th)
    sum_patches = np.sum(hit_patches, axis=(1,2))
    
    avg_patch = np.mean(hit_patches, axis=0)

    # Plot the averaged Gaussian
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    imax = ax.imshow(avg_patch, vmin=avg_patch.min(), vmax=avg_patch.max(), cmap='jet')
    fig.colorbar(imax, ax=ax)
    print(avg_patch)

In [ ]:
# Plot the Landau distribution of patch summed charge
sum_patches = np.sum(hit_patches, axis=(1,2))
qmax = 300
mpv = 19.8
eta = 4.8
A = 0.032 #0.0388
x_landau = np.arange(0, qmax, 0.01)
y_landau = pylandau.landau(x_landau, mpv, eta, A)

plt.figure()
plt.hist(sum_patches[sum_patches < qmax],bins=100,range=[0,qmax],density=True)
plt.plot(x_landau,y_landau,color='red',linewidth=2,label=f"L(mpv={mpv},$\eta$={eta},A={A})")
plt.legend(fontsize=12)
plt.xlabel('Patch charge',fontsize=14)
plt.ylabel('Counts/bin',fontsize=14)

In [ ]:
# The average 3x3 single-electron matrix
matrix = np.array(avg_patch)

# Gaussian function (assuming isotropic and centered at (1,1))
def gaussian(x, y, A, sigma):
    return A * np.exp(-((x-1)**2 + (y-1)**2) / (2*sigma**2))

# Objective function to minimize (MSE)
def objective(params):
    A, sigma = params
    predicted = np.array([[gaussian(x, y, A, sigma) for y in range(3)] for x in range(3)])
    return np.mean((matrix - predicted)**2)

# Initial guesses for A and sigma
initial_guess = [35, 1.3]

# Perform the optimization
result = minimize(objective, initial_guess, method='L-BFGS-B', bounds=[(0, None), (0, None)])

# Optimized parameters
A_opt, sigma_opt = result.x
print(f"Optimized A: {A_opt}, Optimized sigma: {sigma_opt}")

In [ ]:
# Generate the optimized Gaussian 3x3 matrix
optimized_patch = np.array([[gaussian(x, y, A_opt, sigma_opt) for y in range(3)] for x in range(3)])

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Original averaged patch
ax0 = axs[0].imshow(avg_patch, cmap='viridis')
fig.colorbar(ax0, ax=axs[0])
axs[0].set_title('Original Avg. Patch')
axs[0].axis('off')

# Optimized Gaussian patch
ax1 = axs[1].imshow(optimized_patch, cmap='viridis')
fig.colorbar(ax1, ax=axs[1])
axs[1].set_title('Optimized Gaussian Patch')
axs[1].axis('off')

# Difference
ax2 = axs[2].imshow(avg_patch - optimized_patch, cmap='viridis')
fig.colorbar(ax2, ax=axs[2])
axs[2].set_title('Difference (Original - Optimized)')
axs[2].axis('off')

plt.show()

# Print optimized Gaussian
print(optimized_patch)

In [ ]:
# Look at histogram of some frames
nframes = 10
file_path = f"/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan000000{scan_no}.h5"
with h5py.File(file_path, 'r') as f0:
    data = f0['frames']
    
    data_hist = np.array(data[0:nframes,:,:],dtype=np.float32) - 20
    
    plt.hist(data_hist.flatten(), bins=100) #, range=[0,20])
    plt.yscale('log')
    plt.xlabel("baseline subtracted (-20) pixel value")

# Attempt to count a single frame

In [ ]:
A = 31.3
sigma = 1.3
noise_sigma = 1.0

def gaussian_5x5(A, sigma):
    """Generate a 5x5 Gaussian splash."""
    x = y = np.arange(0, 5) - 2  # Centered at 0, 0
    X, Y = np.meshgrid(x, y)
    gaussian = A * np.exp(- (X**2 + Y**2) / (2 * sigma**2))
    return gaussian

# Construct the 5x5 Gaussian
gaus_elec = gaussian_5x5(A, sigma)

Obtain the frame to be counted

In [ ]:
nreco = 1500
baseline = 20.0
th_single_elec = 10

#file_path = f"/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan000000{scan_no}.h5"
file_path = "/pscratch/sd/j/jrenner1/data_scan0000000159.h5"
with h5py.File(file_path, 'r') as f0:
    data = f0['frames']
    
    # Get the frame
    frame = np.array(data[nreco,:,:],dtype=np.float32)
    
    # Get a "prior" counted frame
    frame_ct = (frame - baseline)
    frame_ct[frame_ct < th_single_elec] = 0
    frame_ct = np.rint(frame_ct / A)


fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs = axs.flatten()

# Plotting on the ith subplot
ax0 = axs[0].imshow(frame[260:280,270:290], vmin=frame.min(), vmax=frame.max())
fig.colorbar(ax0, ax=axs[0])
ax1 = axs[1].imshow(frame_ct[260:280,270:290], vmin=frame_ct.min(), vmax=frame_ct.max())
fig.colorbar(ax1, ax=axs[1])

# Print nonzero prior values
print("Nonzero values in frame_ct:")
print(frame_ct[np.nonzero(frame_ct)])

Construct the modeled frame

In [ ]:
def construct_modeled_frame_from_counts(frame_ct, splash):
    modeled_frame = np.zeros(frame_ct.shape)
    nrows, ncols = frame_ct.shape
    splash_size = splash.shape[0]  # Assuming splash is square

    for x in range(nrows):
        for y in range(ncols):
            count = frame_ct[x, y]
            if count > 0:
                # Determine the indices for adding the splash
                x_start = max(0, x - splash_size // 2)
                x_end = min(nrows, x + splash_size // 2 + 1)
                y_start = max(0, y - splash_size // 2)
                y_end = min(ncols, y + splash_size // 2 + 1)
                
                # Adjust splash dimensions if near edges
                splash_x_start = 0 - min(0, x - splash_size // 2)
                splash_y_start = 0 - min(0, y - splash_size // 2)
                splash_x_end = splash.shape[0] + min(0, nrows - (x + splash_size // 2 + 1))
                splash_y_end = splash.shape[1] + min(0, ncols - (y + splash_size // 2 + 1))
                #print(f"x from {x_start} to {x_end}")
                #print(f"splash from {splash_x_start} to {splash_x_end}")

                # Add the splash for each count
                modeled_frame[x_start:x_end, y_start:y_end] += splash[splash_x_start:splash_x_end, splash_y_start:splash_y_end] * count

    return modeled_frame

In [ ]:
modeled_frame = construct_modeled_frame_from_counts(frame_ct, gaus_elec) + np.random.normal(0, noise_sigma, frame.shape)

In [ ]:
plt.hist(modeled_frame.flatten(),bins=100)
plt.yscale('log')

# Creation of averaged frames

In [ ]:
#scan_nos = [1531, 1532, 1533, 1534] # 70 pA
scan_nos = [1543, 1544, 1545, 1546] # 560 pA
nframes = 16512
frame_width = 576
baselines = [20., 20., 20., 20.]
th_single_elec = 30

# Set up the figure
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs = axs.flatten()

for i, scan_no in enumerate(scan_nos):
    print(f"Processing scan {scan_no}...")
    file_path = f"/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan000000{scan_no}.h5"
    with h5py.File(file_path, 'r') as f0:
        data = f0['frames']
    
        # Subtract the baseline and threshold.
        sub_frames = (data[0:nframes, :, :] - baselines[i])
        sub_frames[sub_frames < th_single_elec] = 0
        
        # Compute the sum of all frames.
        avg_frame = np.average(sub_frames,axis=0)
        
        # Plotting on the ith subplot
        imax = axs[i].imshow(avg_frame, vmin=avg_frame.min(), vmax=avg_frame.max(), cmap='gray')
        fig.colorbar(imax, ax=axs[i])
        axs[i].set_title(f'Scan No: {scan_no}')

---

# Analysis of counted data

In [ ]:
!ls /global/cfs/cdirs/ncemhub/distiller/counted/2023.09.18/
# 1547 is first logo scan

In [ ]:
# Function to convert linear indices to 2D (row, col) indices
def linear_to_2d_indices(linear_indices, cols):
    rows = linear_indices // cols
    cols = linear_indices % cols
    return rows, cols

# Function to create a 2D image from the counted frame data
def create_counted_image(frame_data, frame_height, frame_width):
    # Initialize a 2D array with zeros
    image = np.zeros((frame_height, frame_width), dtype=np.int32)

    # Convert linear indices to 2D indices
    rows, cols = linear_to_2d_indices(frame_data, frame_width)

    # Set the hit pixels
    image[rows, cols] = 1

    return image

In [ ]:
scan_groups = [[1531, 1532, 1533, 1534],  # 70 pA, 300 kV, 30 mrad
               [1535, 1536, 1537, 1538],  # 140 pA
               [1539, 1540, 1541, 1542],  # 280 pA
               [1543, 1544, 1545, 1546]]  # 560 pA
clength = [68, 54, 43, 34]  # in mm
idbase = [12284, 12288, 12292, 12296]
nframes = 1
frame_width = 576

# Set up the figure
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs = axs.flatten()

counted_electrons = []
igroup = 3
for i, scan_no in enumerate(scan_groups[igroup]):
    print(f"Processing scan {scan_no}...")
    file_path = f"/global/cfs/cdirs/ncemhub/distiller/counted/2023.09.18/data_scan{scan_no}_id{idbase[igroup]+i}_electrons.h5"
    with h5py.File(file_path, 'r') as f0:

        summed_counted_image = np.zeros((frame_width, frame_width), dtype=np.int32)
        nelec = 0
        for iframe in range(nframes):
            #if(iframe % int(nframes/10) == 0):
            #    print(f"Processing frame {iframe}/{nframes}")
            frame_data = np.array(f0['electron_events']['frames'][iframe])

            # Create a 2D image of the counted frame
            counted_image = create_counted_image(frame_data, frame_width, frame_width)

            # Sum the counted images
            summed_counted_image += counted_image
            nelec += len(frame_data)
            
        # Sum all the counts.
        print(f"-- Total counts: {np.sum(summed_counted_image)}")
        print(f"-- Total nelec:  {nelec}")
        counted_electrons.append(np.sum(summed_counted_image))

        # Plot the summed, counted image
        imax = axs[i].imshow(summed_counted_image, vmin=summed_counted_image.min(), vmax=summed_counted_image.max(), cmap='grey')
        #imax = axs[i].imshow(summed_counted_image[200:250,200:250], vmin=summed_counted_image.min(), vmax=summed_counted_image.max(), cmap='grey')
        fig.colorbar(imax, ax=axs[i])
        axs[i].set_title(f'Scan No: {scan_no}')

# Adjust layout to prevent overlap
plt.tight_layout()

plt.figure(figsize=(8, 6))  # Figure for the sum vs. camera length plot
plt.plot(clength, counted_electrons, marker='o', linestyle='-')
plt.xlabel('Camera length (mm)')
plt.ylabel('Sum of all counted electrons')
plt.grid(True)

---

-----

# Analysis of raw data

In [ ]:
def create_circular_mask(h, w, center=None, radius=None):
    if center is None:  # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None:  # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [ ]:
scan_nos = [1531, 1532, 1533, 1534]
clength = [60, 54, 43, 34]  # in mm
center_pixels = [(280,280), (270,275), (280,288), (271,273)]
radii = [280, 233, 185, 149]
nframes = 16512
frame_width = 576
#baselines = [20., 20., 20., 20.]
baselines = [20.52008230983527, 20.533552410368216, 20.529735138081396, 20.43255047238372]
sums, means, medians, sigmas = [], [], [], []

# Set up the figure
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs = axs.flatten()

for i, scan_no in enumerate(scan_nos):
    print(f"Processing scan {scan_no}...")
    file_path = f"/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan000000{scan_no}.h5"
    with h5py.File(file_path, 'r') as f0:
        data = f0['frames']
    
        # Subtract the baseline and threshold.
        sub_frames = (data[0:nframes, :, :] - baselines[i])
        #sub_frames[sub_frames < 30] = 0
        
        # Compute the sum of all frames.
        sum_frame = sub_frames.sum(axis=0)
        
        # Compute the baseline using the sum of all frames.
        new_baseline = sum_frame[-100:,-100:].sum()/(100**2*nframes)
        print(f"-- new computed baseline = {new_baseline}")

        # Create the circular mask
        mask = create_circular_mask(frame_width, frame_width, center=center_pixels[i], radius=radii[i])
        
        # Apply the mask and sum the pixel values
        masked_sum = sum_frame * mask
        circle_sum = masked_sum.sum()
        sums.append(circle_sum)  # Store the sum of the circular area
        #sums.append(sum_frame.sum())  # Store the sum of all pixels
        
        # Apply the mask to each frame and compute individual sums.
        masked_individual_frames = sub_frames*mask
        sum_individual_frames = masked_individual_frames.sum(axis=(1,2))
        means.append(np.mean(sum_individual_frames))
        medians.append(np.median(sum_individual_frames))
        sigmas.append(np.std(sum_individual_frames))

        # Plotting on the ith subplot
        #imax = axs[i].imshow(sum_frame, vmin=sum_frame.min(), vmax=sum_frame.max())
        imax = axs[i].imshow(masked_sum, vmin=sum_frame.min(), vmax=sum_frame.max())
        fig.colorbar(imax, ax=axs[i])
        axs[i].set_title(f'Scan No: {scan_no}')

# Adjust layout to prevent overlap
plt.tight_layout()

plt.figure(figsize=(8, 6))  # Figure for the sum vs. camera length plot
plt.plot(radii, sums, marker='o', linestyle='-')
#plt.xlabel('Camera Length (mm)')
plt.xlabel('Radius of integration area (pixels)')
plt.ylabel('Sum of All Pixels')
plt.grid(True)

plt.figure(figsize=(8, 6))  # Figure for the means
#plt.errorbar(radii, means, yerr=np.sqrt(means), marker='o', linestyle='-')
plt.plot(radii, means, marker='o', linestyle='-', color='blue', label='Mean')
plt.plot(radii, medians, marker='o', linestyle='-', color='green', label='Median')
plt.legend()
#plt.xlabel('Camera Length (mm)')
plt.xlabel('Radius of integration area (pixels)')
plt.ylabel('Mean/median pixel sum in each frame')
plt.grid(True)

plt.show()

---

In [ ]:
# Create the "prior"
with h5py.File(dataset_path, 'r') as f0:
    data = f0['frames']
    
    # Get all frames and subtract the baseline.
    prior_bls = np.array(data[:,:,:],dtype=np.float32) - baseline
    
    # Compute the summed frame.
    prior_frame = np.sum(prior_bls,axis=0)
    
    # Divide by the average electron amplitude.
    prior_frame /= A

---

### Test one scan

In [ ]:
scan_no = 1533
nframes = 16512
file_path = f"/global/cfs/cdirs/ncemhub/distiller/raw/2023.09.18/data_scan000000{scan_no}.h5"
f0 = h5py.File(file_path, 'r')
data = f0['frames']

initial_data = data[0:1000, :, :].sum(axis=0)
fg1, axes = plt.subplots(2, 2)
imax1 = axes[0,0].imshow(initial_data, vmin=initial_data[0:288,:].min(), vmax=initial_data[50:-50, 50:-50].max())
plt.colorbar(imax1)